# 导入库

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2

# 准备训练数据 

In [3]:
label_name=pd.read_csv('train/train.csv')
label_name=np.array(label_name)
image_name=label_name[:,0]
labels=label_name[:,1]

In [11]:
images=[]
for i in image_name:
    image=cv2.imread('train/data/'+i)
    images.append(image)

In [ ]:
for i in range(len(images)):
    if images[i] is None:
        images[i] = images[1]
    images[i]=cv2.resize(images[i],(300,300))

In [75]:
images = np.array(images)
labels = np.array(labels)

# 导入keras库

In [67]:
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras import backend as K
from keras.layers import Input
from keras.preprocessing import image
from keras.callbacks import TensorBoard

C:\Users\jerry\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [76]:
from sklearn.cross_validation import train_test_split
import random

X_train, X_valid, y_train, y_valid = train_test_split(images, labels, test_size=0.15, random_state=random.randint(0,100))

# 建立模型

In [69]:
inputs = Input(shape=(300,300,3))
tower_1 = Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
tower_1 = Conv2D(32, (3, 3), activation='relu')(tower_1)
tower_1 = MaxPooling2D(pool_size=(2, 2))(tower_1)
tower_1 = Dropout(0.25)(tower_1)

tower_2 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)
tower_2 = Conv2D(64, (3, 3), activation='relu')(tower_2)
tower_2 = MaxPooling2D(pool_size=(2,2))(tower_2)
tower_2 = Dropout(0.25)(tower_2)

tower_3 = Flatten()(tower_2)
tower_3 = Dense(512, activation='relu')(tower_3)
tower_3 = Dropout(0.5)(tower_3)
predictions = Dense(1, activation='sigmoid')(tower_3)

model = Model(inputs=inputs, outputs=predictions)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [70]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300, 300, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 300, 300, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 298, 298, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 149, 149, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 149, 149, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 147, 147, 64)      36928     
__________

# 训练模型

In [ ]:
model.fit(X_train,y_train,
          batch_size=32,
          epochs=10,
          validation_split=0.2,
          #validation_data=(X_valid, y_valid),
          callbacks=[TensorBoard(log_dir='./tmp/log')],
          shuffle=True)

In [ ]:
for i 